<h1>Содержание<span class="tocSkip"></span></h1>
<br>
<div class="toc">
</div>

# Лабораторная работа №2: Pygmo

**Задача:** оптимизировать алгоритмы градиентного спуска с помощью Pygmo.

**Источник данных:** набор данных взят с платформы [Kaggle](https://www.kaggle.com/datasets/rounakbanik/pokemon).

**Описание данных:** набор данных содержит информацию обо всех семи поколениях 802 Покемонов.

---

Для реализации поставленных задач из набора данных `datasets/pokemon.csv` будут взяты следующие столбцы:

* `defense` - признак объектов
* `attack` - целевой признак
* `name` - имя Покемона

<div style="height: 2px; background-color: blue; margin: 10px 0;"></div>

## 1. Импорт библиотек

In [ ]:
import pygmo

In [ ]:
from PyGMO.problem import base

class my_problem(base):
    """
    De Jong (sphere) function implemented purely in Python.

    USAGE: my_problem(dim=10)

    * dim problem dimension
    """

    def __init__(self, dim=10):
        # First we call the constructor of the base class telling PyGMO
        # what kind of problem to expect ('dim' dimensions, 1 objective, 0 contraints etc.)
        super(my_problem,self).__init__(dim)

        # We set the problem bounds (in this case equal for all components)
        self.set_bounds(-5.12, 5.12)

    # Reimplement the virtual method that defines the objective function.
    def _objfun_impl(self, x):

        # Compute the sphere function
        f = sum([x[i] ** 2 for i in range(self.dimension)])

        # Note that we return a tuple with one element only. In PyGMO the objective functions
        # return tuples so that multi-objective optimization is also possible.
        return (f, )

    # Finally we also reimplement a virtual method that adds some output to the __repr__ method
    def human_readable_extra(self):
        return "\n\t Problem dimension: " + str(self.__dim)

In [ ]:
from PyGMO import algorithm, island

prob = my_problem(dim=10)  # Create a 10-dimensional problem
algo = algorithm.bee_colony(gen=500)  # 500 generations of bee_colony algorithm
isl = island(algo, prob, 20)  # Instantiate population with 20 individuals
isl.evolve(1)  # Evolve the island once
isl.join()
print(isl.population.champion.f)

In [ ]:
class my_problem_max(base):
    """
    Analytical function to maximize.

    USAGE: my_problem_max()
    """

    def __init__(self):
        super(my_problem_max,self).__init__(2)
        self.set_bounds(-10, 10)

        # We provide a list of the best known solutions to the problem
        self.best_x = [[1.0, -1.0], ]

    # Reimplement the virtual method that defines the objective function
    def _objfun_impl(self, x):
        f = -(1.0 - x[0]) ** 2 - 100 * (-x[0] ** 2 - x[1]) ** 2 - 1.0
        return (f, )

    # Reimplement the virtual method that compares fitnesses
    def _compare_fitness_impl(self, f1, f2):
        return f1[0] > f2[0]

    # Add some output to __repr__
    def human_readable_extra(self):
        return "\n\tMaximization problem"

In [ ]:
from math import sqrt

prob = my_problem_max()
algo = algorithm.de(gen=20)
isl = island(algo, prob, 20)
isl.evolve(10)
isl.join()

print("Best individual:")
print(isl.population.champion)

print("Comparison of the best found fitness with the best known fitness:")
for best_fitness in prob.best_f:
    print(best_fitness[0] - isl.population.champion.f[0])

print("L2 distance to the best decision vector:")
for best_decision in prob.best_x:
    l2_norm = 0
    for n in range(0, len(best_decision)):
        l2_norm +=  (best_decision[n] - isl.population.champion.x[n]) ** 2
    l2_norm = sqrt(l2_norm)
    print(l2_norm)

In [ ]:
class my_mo_problem(base):
    """
    A multi-objective problem.
    (This is actually a Python implementation of 2-dimensional ZDT-1 problem)

    USAGE: my_mo_problem()
    """

    def __init__(self, dim=2):
        # We call the base constructor as 'dim' dimensional problem, with 0 integer parts and 2 objectives.
        super(my_mo_problem,self).__init__(dim, 0, 2)
        self.set_bounds(0.0, 1.0)

    # Reimplement the virtual method that defines the objective function
    def _objfun_impl(self, x):
        f0 = x[0]
        g = 1.0 + 4.5 * x[1]
        f1 = g * (1.0 - sqrt(f0 / g))
        return (f0, f1, )

    # Add some output to __repr__
    def human_readable_extra(self):
        return "\n\tMulti-Objective problem"

In [ ]:
from PyGMO import population

prob = my_mo_problem()
algo = algorithm.sms_emoa(gen=2000)  # 2000 generations of SMS-EMOA should solve it
pop = population(prob, 30)
pop = algo.evolve(pop)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Fold each objectives into vectors and print the Pareto front
F = np.array([ind.cur_f for ind in pop]).T
plt.scatter(F[0], F[1])
plt.xlabel("$f^{(1)}$")
plt.ylabel("$f^{(2)}$")
plt.show()

<div style="text-align: center; font-size: 20px; padding: 15px 0;">
    <a href="#Содержание" data-toc-modified-id="Содержание" style="text-decoration: none; color: #296eaa; border: 2px dashed #296eaa; opacity: 0.8; border-radius: 3px; padding: 10px 80px;">
        Наверх к содержанию ↑
    </a>
</div>